### Load B.S. Detector "Fake news" dataset

We will follow the same basic idea we used for the "Liar, Liar" dataset

In [1]:
import pandas as pd

bs_raw = pd.read_csv("./fake-news/fake.csv")

bs_raw.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [2]:
len(bs_raw)

12999

In [3]:
pd.value_counts(bs_raw['language'])

english       12403
russian         203
spanish         172
german          111
french           38
arabic           22
portuguese       11
turkish          10
italian           9
ignore            7
greek             4
norwegian         3
dutch             3
chinese           1
polish            1
finnish           1
Name: language, dtype: int64

Immediately we see that we have a problem with this dataset as we planned to use it: it's made up entirely of "fake news," with no "real news" examples. As such, it may be suitable for unsupervised learning, but on its own we can't use this for training a supervised model. We will still load it in and tokenize it, and will have to decide whether and how we will use this.

We also have a lot of missing values. If we want to preserve the matching between headlines and text, we'll have to drop records where either is missing. 

There are also some examples that are not in English. This may confuse our model, so we will only keep the English language examples, which make up the vast majority of the data.

In [4]:
bs_english = bs_raw.loc[bs_raw["language"] == "english"]
bs_keep = bs_english[["title","text"]].dropna()


In [5]:
bs_keep.head()

,title,text
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...


In [6]:
bs_headlines = list(bs_keep["title"])
bs_text = list(bs_keep["text"])

In [7]:
print(len(bs_headlines), len(bs_text))

11677 11677


After removing missing data and non-English examples, we have 11,677 remaining pairs of headlines and text. 

As before, let's look at some random examples to familiarize ourselves with the data.

In [8]:
# Again, look at some random examples
import random
ex_num = random.randint(1, len(bs_headlines))

print(bs_headlines[ex_num]) 
print(bs_text[ex_num])

Fifty interesting facts about Russia
Fifty interesting facts about Russia 31.10.2016 Print version Font Size 1. Russia is the largest country in the world. The country has ​​17,075,400 square kilometers, which makes Russia 1.8 times larger than the United States of America. The territory that Russia takes on planet Earth practically equals the surface area of ​​planet Pluto.2. Russia is home to the world's largest active volcano - Klyuchevskaya Sopka. Its height is 4 kilometers 850 meters. The volcano shoots columns of ash up to eight kilometers high and becomes even taller with every eruption. Klyuchevskaya Sopka volcano erupts for the past 7,000 years.3. The subway system (metro) of St. Petersburg is the deepest in the world. Its average depth - 100 meters.4. The number of bridges in St. Petersburg exceeds the number of bridges in Venice three times. 5. The oldest Christian church on the territory of Russia is the ancient temple of Thaba-Yerdy located in Ingushetia, Jeirakh District.

Some of these text examples are very long, much longer than the statements in the "liar, liar" dataset. There are also some that are very short.

Finally, let's tokenize our headlines and text, and look at a tokenized example.

In [9]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split(' ') for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type '+token)
        
bs_head_tokens = tokenize(bs_headlines, token = 'word')
bs_text_tokens = tokenize(bs_text, token = 'word')

In [10]:
bs_head_tokens[ex_num]

['Fifty', 'interesting', 'facts', 'about', 'Russia']

In [11]:
bs_text_tokens[ex_num]

['Fifty',
 'interesting',
 'facts',
 'about',
 'Russia',
 '31.10.2016',
 'Print',
 'version',
 'Font',
 'Size',
 '1.',
 'Russia',
 'is',
 'the',
 'largest',
 'country',
 'in',
 'the',
 'world.',
 'The',
 'country',
 'has',
 '\u200b\u200b17,075,400',
 'square',
 'kilometers,',
 'which',
 'makes',
 'Russia',
 '1.8',
 'times',
 'larger',
 'than',
 'the',
 'United',
 'States',
 'of',
 'America.',
 'The',
 'territory',
 'that',
 'Russia',
 'takes',
 'on',
 'planet',
 'Earth',
 'practically',
 'equals',
 'the',
 'surface',
 'area',
 'of',
 '\u200b\u200bplanet',
 'Pluto.2.',
 'Russia',
 'is',
 'home',
 'to',
 'the',
 "world's",
 'largest',
 'active',
 'volcano',
 '-',
 'Klyuchevskaya',
 'Sopka.',
 'Its',
 'height',
 'is',
 '4',
 'kilometers',
 '850',
 'meters.',
 'The',
 'volcano',
 'shoots',
 'columns',
 'of',
 'ash',
 'up',
 'to',
 'eight',
 'kilometers',
 'high',
 'and',
 'becomes',
 'even',
 'taller',
 'with',
 'every',
 'eruption.',
 'Klyuchevskaya',
 'Sopka',
 'volcano',
 'erupts',
 'fo

### Vocab and Word Embeddings

We'll use the word embedding similar to the Liar, Liar dataset.

In [12]:
from mxnet import gluon, init, np, npx
from mxnet.gluon import nn, rnn
from mxnet.contrib import text
npx.set_np()

In [16]:
# TODO(JC): I'm trying to avoid calling the d2l package directly, but I'm having a hard time  
#           adapting the Vocab function from the version defined in d2l 8.2

import d2l
vocab = d2l.Vocab(bs_text_tokens, min_freq=5, reserved_tokens=['<pad>']) 
    #JC: playing with min_freq, as our vocab is pretty small

In [14]:
# Load pre-trained GloVe word embedding. This is available as a resource in mxnet
glove_embedding = text.embedding.create(
    'glove', pretrained_file_name='glove.6B.100d.txt')

In [17]:
embeds = glove_embedding.get_vecs_by_tokens(vocab.idx_to_token)
embeds.shape
# This vocabulary is significantly larger than the one from the liar, liar dataset.

(68291, 100)

In [ ]:
# JC: Next step: ???